In [1]:
import tvm
import os
import numpy as np
from tvm.relay.testing import run_opt_pass
from tvm import relay
from tvm.relay import transform, build_module
from tensorflow import keras
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3' 
os.environ['TF_FORCE_GPU_ALLOW_GROWTH'] = "true"

In [29]:
from tvm.relay.dataflow_pattern import *

class QuantCallback(DFPatternCallback):
    # A callback class to rewrite the matched pattern to a batch_norm op.
    def __init__(self, name_hints, require_type=False):
        super().__init__(require_type)
        super().__init__(rewrite_once=True)

        # self.tuple_get_item_node = is_tuple_get_item(wildcard(), 0)
        # self.pattern_1 = self.tuple_get_item_node
        # leaky_relu_node = is_op('nn.leaky_relu')(wildcard())| is_op('nn.relu')(wildcard())
        # self.name_hints = name_hints
        # print(name_hints[0])
        # self.var = is_var(str(name_hints[0]))
        self.var = is_expr(name_hints[0])
        # self.param = wildcard()
        # self.pattern = is_op("nn.conv2d")(self.var, self.param)
        self.pattern = self.var
        self.counter = 0
        self.tmp = []

    def quant(self, node):
        # cast_to_int8 = relay.cast(
        #     relay.clip(
        #         relay.round(
        #             relay.multiply(node, relay.const(8.0))
        #         ), 
        #         a_min=-127.0, a_max=127.0
        #     ),
        #     dtype="int8"
        # )

        cast_to_int8 = relay.cast(
            relay.clip(
                relay.round(
                    relay.multiply(
                        relay.subtract(node, relay.const(18.0))
                        , relay.const(7.0))
                ), 
                a_min=-127.0, a_max=127.0
            ),
            dtype="int8"
        )
        # result_node = relay.annotation.stop_fusion(cast_to_int8)
        # self.tmp.append(result_node)
        return cast_to_int8

    def dequant(self, node):
        # cast_to_float32 = relay.divide(
        #     relay.cast(node, dtype='float32'), relay.const(8.0)
        # )
        cast_to_float32 = relay.add(
                relay.divide(
                relay.cast(node, dtype='float32'), relay.const(7.0))
            , relay.const(18.0)
        )
        return cast_to_float32

    def callback(self, pre, post, node_map):
        print("quant")
        a = node_map[self.var][0]
        print(pre)
        print(a)
        try:
            if a.name_hint in self.name_hints:
                print("asdasd")
                new_var = relay.var('input_0', relay.TensorType(a.checked_type.shape, 'int8'))
                return relay.nn.conv2d(self.dequant(new_var), b)
        except Exception as e:
            print(e)
            return post
        return post


In [3]:
model = keras.models.load_model("./simple_model.h5")
data = np.random.randint(0, 256, size=(1, 256,256,3)) / 255
data = data.transpose([0, 3, 1, 2])

shape_dict = {"input_1": data.shape}

mod, params = relay.frontend.from_keras(model, shape_dict)

In [15]:
anf = run_opt_pass(mod['main'], transform.ToANormalForm())
anf = run_opt_pass(anf, transform.InferType())

print(anf)

fn (%input_1: Tensor[(1, 3, 256, 256), float32] /* ty=Tensor[(1, 3, 256, 256), float32] */, %v_param_1: Tensor[(16, 3, 3, 3), float32] /* ty=Tensor[(16, 3, 3, 3), float32] */, %v_param_2: Tensor[(16), float32] /* ty=Tensor[(16), float32] */) -> Tensor[(1, 16, 128i64, 128i64), float32] {
  let %x_3: Tensor[(1, 16, 256, 256), float32] /* ty=Tensor[(1, 16, 256, 256), float32] */ = nn.conv2d(%input_1, %v_param_1, padding=[1i64, 1i64, 1i64, 1i64], channels=16, kernel_size=[3, 3]) /* ty=Tensor[(1, 16, 256, 256), float32] */;
  let %x_4: Tensor[(1, 16, 256, 256), float32] /* ty=Tensor[(1, 16, 256, 256), float32] */ = nn.bias_add(%x_3, %v_param_2) /* ty=Tensor[(1, 16, 256, 256), float32] */;
  let %x_5: Tensor[(1, 16, 128i64, 128i64), float32] /* ty=Tensor[(1, 16, 128i64, 128i64), float32] */ = nn.max_pool2d(%x_4, pool_size=[2, 2], strides=[2, 2], padding=[0i64, 0i64, 0i64, 0i64]) /* ty=Tensor[(1, 16, 128i64, 128i64), float32] */;
  %x_5
} /* ty=fn (Tensor[(1, 3, 256, 256), float32], Tensor[(1

In [7]:
qcb = QuantCallback(['input_1'])

In [30]:
qcb = QuantCallback([relay.analysis.free_vars(anf.body)[0]])

In [31]:
newanf = rewrite(qcb, anf.body)

In [92]:
relay.analysis.free_vars(newanf)

[Var(input_0, ty=TensorType([1, 3, 256, 256], int8))]

In [109]:
def dequant(node):
    # cast_to_float32 = relay.divide(
    #     relay.cast(node, dtype='float32'), relay.const(8.0)
    # )
    cast_to_float32 = relay.add(
            relay.divide(
            relay.cast(node, dtype='float32'), relay.const(7.0))
        , relay.const(18.0)
    )
    # cast_to_float32 = run_opt_pass(cast_to_float32, transform.ToANormalForm())
    # cast_to_float32 = run_opt_pass(cast_to_float32, transform.InferType())
    return cast_to_float32

In [2]:
newf = relay.Function(relay.analysis.free_vars(newanf), anf.body)

NameError: name 'newanf' is not defined

In [43]:
print(anf.body.astext(False))

#[version = "0.0.5"]
free_var %input_1: Tensor[(1, 3, 256, 256), float32] /* ty=Tensor[(1, 3, 256, 256), float32] */;
free_var %v_param_1: Tensor[(16, 3, 3, 3), float32] /* ty=Tensor[(16, 3, 3, 3), float32] */;
let %x_3: Tensor[(1, 16, 256, 256), float32] /* ty=Tensor[(1, 16, 256, 256), float32] */ = nn.conv2d(%input_1, %v_param_1, padding=[1i64, 1i64, 1i64, 1i64], channels=16, kernel_size=[3, 3]) /* ty=Tensor[(1, 16, 256, 256), float32] */;
free_var %v_param_2: Tensor[(16), float32] /* ty=Tensor[(16), float32] */;
let %x_4: Tensor[(1, 16, 256, 256), float32] /* ty=Tensor[(1, 16, 256, 256), float32] */ = nn.bias_add(%x_3, %v_param_2) /* ty=Tensor[(1, 16, 256, 256), float32] */;
let %x_5: Tensor[(1, 16, 128i64, 128i64), float32] /* ty=Tensor[(1, 16, 128i64, 128i64), float32] */ = nn.max_pool2d(%x_4, pool_size=[2, 2], strides=[2, 2], padding=[0i64, 0i64, 0i64, 0i64]) /* ty=Tensor[(1, 16, 128i64, 128i64), float32] */;
%x_5


In [41]:
anf.body

LetNode(Var(x_3, ty=TensorType([1, 16, 256, 256], float32)), CallNode(Op(nn.conv2d), [Var(input_1, ty=TensorType([1, 3, 256, 256], float32)), Var(_param_1, ty=TensorType([16, 3, 3, 3], float32))], relay.attrs.Conv2DAttrs(0x2ff24698), [TensorType([1, 3, 256, 256], float32), TensorType([16, 3, 3, 3], float32)]), LetNode(Var(x_4, ty=TensorType([1, 16, 256, 256], float32)), CallNode(Op(nn.bias_add), [Var(x_3, ty=TensorType([1, 16, 256, 256], float32)), Var(_param_2, ty=TensorType([16], float32))], relay.attrs.BiasAddAttrs(0x2ff52ec8), [TensorType([1, 16, 256, 256], float32), TensorType([16], float32)]), LetNode(Var(x_5, ty=TensorType([1, 16, (int64)128, (int64)128], float32)), CallNode(Op(nn.max_pool2d), [Var(x_4, ty=TensorType([1, 16, 256, 256], float32))], relay.attrs.MaxPool2DAttrs(0x2ff25db8), [TensorType([1, 16, 256, 256], float32)]), Var(x_5, ty=TensorType([1, 16, (int64)128, (int64)128], float32)))))

In [98]:
anf.body.value.args[0]

Var(input_1, ty=TensorType([1, 3, 256, 256], float32))

In [122]:
anf = run_opt_pass(anf, transform.ToGraphNormalForm())


In [124]:
print(anf.body.astext(False))

#[version = "0.0.5"]
free_var %input_1: Tensor[(1, 3, 256, 256), float32] /* ty=Tensor[(1, 3, 256, 256), float32] */;
free_var %v_param_1: Tensor[(16, 3, 3, 3), float32] /* ty=Tensor[(16, 3, 3, 3), float32] */;
%0 = nn.conv2d(%input_1, %v_param_1, padding=[1i64, 1i64, 1i64, 1i64], channels=16, kernel_size=[3, 3]) /* ty=Tensor[(1, 16, 256, 256), float32] */;
free_var %v_param_2: Tensor[(16), float32] /* ty=Tensor[(16), float32] */;
%1 = nn.bias_add(%0, %v_param_2) /* ty=Tensor[(1, 16, 256, 256), float32] */;
nn.max_pool2d(%1, pool_size=[2, 2], strides=[2, 2], padding=[0i64, 0i64, 0i64, 0i64]) /* ty=Tensor[(1, 16, 128i64, 128i64), float32] */


In [121]:
print(new_anf.astext(False))

#[version = "0.0.5"]
free_var %input_0: Tensor[(1, 3, 256, 256), int8] /* ty=Tensor[(1, 3, 256, 256), int8] */;
%0 = cast(%input_0, dtype="float32") /* ty=Tensor[(1, 3, 256, 256), float32] */;
%1 = divide(%0, 7f /* ty=float32 */) /* ty=Tensor[(1, 3, 256, 256), float32] */;
%2 = add(%1, 18f /* ty=float32 */) /* ty=Tensor[(1, 3, 256, 256), float32] */;
free_var %v_param_1: Tensor[(16, 3, 3, 3), float32] /* ty=Tensor[(16, 3, 3, 3), float32] */;
%3 = nn.conv2d(%2, %v_param_1, padding=[1i64, 1i64, 1i64, 1i64], channels=16, kernel_size=[3, 3]) /* ty=Tensor[(1, 16, 256, 256), float32] */;
free_var %v_param_2: Tensor[(16), float32] /* ty=Tensor[(16), float32] */;
%4 = nn.bias_add(%3, %v_param_2) /* ty=Tensor[(1, 16, 256, 256), float32] */;
nn.max_pool2d(%4, pool_size=[2, 2], strides=[2, 2], padding=[0i64, 0i64, 0i64, 0i64]) /* ty=Tensor[(1, 16, 128i64, 128i64), float32] */


In [64]:
tt =dequant(anf.body.value.args[0])

In [79]:
print(tt.astext(False))

#[version = "0.0.5"]
free_var %input_1: Tensor[(1, 3, 256, 256), float32] /* ty=Tensor[(1, 3, 256, 256), float32] */;
let %x_6: Tensor[(1, 3, 256, 256), float32] /* ty=Tensor[(1, 3, 256, 256), float32] */ = cast(%input_1, dtype="float32") /* ty=Tensor[(1, 3, 256, 256), float32] */;
let %x_7: float32 /* ty=float32 */ = 7f /* ty=float32 */;
let %x_8: Tensor[(1, 3, 256, 256), float32] /* ty=Tensor[(1, 3, 256, 256), float32] */ = divide(%x_6, %x_7) /* ty=Tensor[(1, 3, 256, 256), float32] */;
let %x_9: float32 /* ty=float32 */ = 18f /* ty=float32 */;
let %x_10: Tensor[(1, 3, 256, 256), float32] /* ty=Tensor[(1, 3, 256, 256), float32] */ = add(%x_8, %x_9) /* ty=Tensor[(1, 3, 256, 256), float32] */;
%x_10


In [97]:
anf.body.var

Var(x_3, ty=TensorType([1, 16, 256, 256], float32))

In [101]:
relay.analysis.free_vars(anf.body)[0]

Var(input_1, ty=TensorType([1, 3, 256, 256], float32))

In [110]:
new_input = tvm.relay.expr.Var('input_0', relay.TensorType(anf.body.value.args[0].checked_type.shape, 'int8'))
tt = dequant(new_input)
new_anf = tvm.relay.expr.Let(relay.analysis.free_vars(anf.body)[0], tt, anf.body)

In [138]:
anf.body

CallNode(Op(nn.max_pool2d), [CallNode(Op(nn.bias_add), [CallNode(Op(nn.conv2d), [Var(input_1, ty=TensorType([1, 3, 256, 256], float32)), Var(_param_1, ty=TensorType([16, 3, 3, 3], float32))], relay.attrs.Conv2DAttrs(0x2ff24698), [TensorType([1, 3, 256, 256], float32), TensorType([16, 3, 3, 3], float32)]), Var(_param_2, ty=TensorType([16], float32))], relay.attrs.BiasAddAttrs(0x2ff52ec8), [TensorType([1, 16, 256, 256], float32), TensorType([16], float32)])], relay.attrs.MaxPool2DAttrs(0x2ff25db8), [TensorType([1, 16, 256, 256], float32)])

In [118]:
mod = new_anf
ann = run_opt_pass(mod, transform.ToGraphNormalForm())
mod = tvm.IRModule.from_expr(ann)['main']

In [119]:
mod

fn (%input_0: Tensor[(1, 3, 256, 256), int8] /* ty=Tensor[(1, 3, 256, 256), int8] */, %v_param_1: Tensor[(16, 3, 3, 3), float32] /* ty=Tensor[(16, 3, 3, 3), float32] */, %v_param_2: Tensor[(16), float32] /* ty=Tensor[(16), float32] */) {
  %0 = cast(%input_0, dtype="float32") /* ty=Tensor[(1, 3, 256, 256), float32] */;
  %1 = divide(%0, 7f /* ty=float32 */) /* ty=Tensor[(1, 3, 256, 256), float32] */;
  %2 = add(%1, 18f /* ty=float32 */) /* ty=Tensor[(1, 3, 256, 256), float32] */;
  %3 = nn.conv2d(%2, %v_param_1, padding=[1i64, 1i64, 1i64, 1i64], channels=16, kernel_size=[3, 3]) /* ty=Tensor[(1, 16, 256, 256), float32] */;
  %4 = nn.bias_add(%3, %v_param_2) /* ty=Tensor[(1, 16, 256, 256), float32] */;
  nn.max_pool2d(%4, pool_size=[2, 2], strides=[2, 2], padding=[0i64, 0i64, 0i64, 0i64]) /* ty=Tensor[(1, 16, 128i64, 128i64), float32] */
}

In [127]:
new_anf = run_opt_pass(new_anf, transform.ToGraphNormalForm())
new_mod = tvm.IRModule.from_expr(new_anf)


In [128]:
def quant(node):
        cast_to_int8 = relay.cast(
            relay.clip(
                relay.round(
                    relay.multiply(
                        relay.subtract(node, relay.const(18.0))
                        , relay.const(7.0))
                ), 
                a_min=-127.0, a_max=127.0
            ),
            dtype="int8"
        )
        # result_node = relay.annotation.stop_fusion(cast_to_int8)
        # self.tmp.append(result_node)
        return cast_to_int8

In [134]:
new_mod

#[version = "0.0.5"]
def @main(%input_0: Tensor[(1, 3, 256, 256), int8] /* ty=Tensor[(1, 3, 256, 256), int8] */, %v_param_1: Tensor[(16, 3, 3, 3), float32] /* ty=Tensor[(16, 3, 3, 3), float32] */, %v_param_2: Tensor[(16), float32] /* ty=Tensor[(16), float32] */) {
  %0 = cast(%input_0, dtype="float32") /* ty=Tensor[(1, 3, 256, 256), float32] */;
  %1 = divide(%0, 7f /* ty=float32 */) /* ty=Tensor[(1, 3, 256, 256), float32] */;
  %2 = add(%1, 18f /* ty=float32 */) /* ty=Tensor[(1, 3, 256, 256), float32] */;
  %3 = nn.conv2d(%2, %v_param_1, padding=[1i64, 1i64, 1i64, 1i64], channels=16, kernel_size=[3, 3]) /* ty=Tensor[(1, 16, 256, 256), float32] */;
  %4 = nn.bias_add(%3, %v_param_2) /* ty=Tensor[(1, 16, 256, 256), float32] */;
  nn.max_pool2d(%4, pool_size=[2, 2], strides=[2, 2], padding=[0i64, 0i64, 0i64, 0i64]) /* ty=Tensor[(1, 16, 128i64, 128i64), float32] */
}

In [135]:
new_func = relay.Function(new_mod['main'].params, quant(new_mod['main'].body))

In [136]:
new_func

fn (%input_0: Tensor[(1, 3, 256, 256), int8] /* ty=Tensor[(1, 3, 256, 256), int8] */, %v_param_1: Tensor[(16, 3, 3, 3), float32] /* ty=Tensor[(16, 3, 3, 3), float32] */, %v_param_2: Tensor[(16), float32] /* ty=Tensor[(16), float32] */) {
  %0 = cast(%input_0, dtype="float32") /* ty=Tensor[(1, 3, 256, 256), float32] */;
  %1 = divide(%0, 7f /* ty=float32 */) /* ty=Tensor[(1, 3, 256, 256), float32] */;
  %2 = add(%1, 18f /* ty=float32 */) /* ty=Tensor[(1, 3, 256, 256), float32] */;
  %3 = nn.conv2d(%2, %v_param_1, padding=[1i64, 1i64, 1i64, 1i64], channels=16, kernel_size=[3, 3]) /* ty=Tensor[(1, 16, 256, 256), float32] */;
  %4 = nn.bias_add(%3, %v_param_2) /* ty=Tensor[(1, 16, 256, 256), float32] */;
  %5 = nn.max_pool2d(%4, pool_size=[2, 2], strides=[2, 2], padding=[0i64, 0i64, 0i64, 0i64]) /* ty=Tensor[(1, 16, 128i64, 128i64), float32] */;
  %6 = subtract(%5, 18f);
  %7 = multiply(%6, 7f);
  %8 = round(%7);
  %9 = clip(%8, a_min=-127f, a_max=127f);
  cast(%9, dtype="int8")
}

In [137]:
with tvm.transform.PassContext(opt_level=4):
    lib = relay.build(new_func, 'cuda', params=params)

In [111]:
print(tt.astext(False))

#[version = "0.0.5"]
free_var %input_0: Tensor[(1, 3, 256, 256), int8];
%0 = cast(%input_0, dtype="float32");
%1 = divide(%0, 7f);
add(%1, 18f)


In [112]:
print(new_anf.astext(False))

#[version = "0.0.5"]
free_var %input_0: Tensor[(1, 3, 256, 256), int8];
%0 = cast(%input_0, dtype="float32");
%1 = divide(%0, 7f);
let %input_1: Tensor[(1, 3, 256, 256), float32] /* ty=Tensor[(1, 3, 256, 256), float32] */ = add(%1, 18f);
free_var %v_param_1: Tensor[(16, 3, 3, 3), float32] /* ty=Tensor[(16, 3, 3, 3), float32] */;
let %x_3: Tensor[(1, 16, 256, 256), float32] /* ty=Tensor[(1, 16, 256, 256), float32] */ = nn.conv2d(%input_1, %v_param_1, padding=[1i64, 1i64, 1i64, 1i64], channels=16, kernel_size=[3, 3]) /* ty=Tensor[(1, 16, 256, 256), float32] */;
free_var %v_param_2: Tensor[(16), float32] /* ty=Tensor[(16), float32] */;
let %x_4: Tensor[(1, 16, 256, 256), float32] /* ty=Tensor[(1, 16, 256, 256), float32] */ = nn.bias_add(%x_3, %v_param_2) /* ty=Tensor[(1, 16, 256, 256), float32] */;
let %x_5: Tensor[(1, 16, 128i64, 128i64), float32] /* ty=Tensor[(1, 16, 128i64, 128i64), float32] */ = nn.max_pool2d(%x_4, pool_size=[2, 2], strides=[2, 2], padding=[0i64, 0i64, 0i64, 0i64]) /*

In [117]:
new_anf

[Var(input_0, ty=TensorType([1, 3, 256, 256], int8)), Var(_param_1, ty=TensorType([16, 3, 3, 3], float32)), Var(_param_2, ty=TensorType([16], float32))]

In [93]:
type(tt.body.body.body.body)

tvm.relay.expr.Let

In [ ]:
tvm.relay.expr.Let(
    anf.var,
    value,
    _dequant(anf.body),
)

In [85]:
print(anf.body.astext(False))

#[version = "0.0.5"]
free_var %input_1: Tensor[(1, 3, 256, 256), float32] /* ty=Tensor[(1, 3, 256, 256), float32] */;
free_var %v_param_1: Tensor[(16, 3, 3, 3), float32] /* ty=Tensor[(16, 3, 3, 3), float32] */;
let %x_3: Tensor[(1, 16, 256, 256), float32] /* ty=Tensor[(1, 16, 256, 256), float32] */ = nn.conv2d(%input_1, %v_param_1, padding=[1i64, 1i64, 1i64, 1i64], channels=16, kernel_size=[3, 3]) /* ty=Tensor[(1, 16, 256, 256), float32] */;
free_var %v_param_2: Tensor[(16), float32] /* ty=Tensor[(16), float32] */;
let %x_4: Tensor[(1, 16, 256, 256), float32] /* ty=Tensor[(1, 16, 256, 256), float32] */ = nn.bias_add(%x_3, %v_param_2) /* ty=Tensor[(1, 16, 256, 256), float32] */;
let %x_5: Tensor[(1, 16, 128i64, 128i64), float32] /* ty=Tensor[(1, 16, 128i64, 128i64), float32] */ = nn.max_pool2d(%x_4, pool_size=[2, 2], strides=[2, 2], padding=[0i64, 0i64, 0i64, 0i64]) /* ty=Tensor[(1, 16, 128i64, 128i64), float32] */;
%x_5


In [53]:
anf.body.value.args[0] = dequant(anf.body.value.args[0])

TypeError: 'Array' object does not support item assignment

In [48]:
type(anf.body.value)

tvm.relay.expr.Call

In [32]:
pat = is_var('input_1')
pat = is_expr(relay.analysis.free_vars(anf.body)[0])

In [17]:
relay.analysis.free_vars(anf.body)

[Var(input_1, ty=TensorType([1, 3, 256, 256], float32)), Var(_param_1, ty=TensorType([16, 3, 3, 3], float32)), Var(_param_2, ty=TensorType([16], float32))]

In [35]:
pat.match(relay.analysis.free_vars(anf.body)[0])

1